In [ ]:
## Convertir datos crudos e inconsistentes en un dataset confiable, estructurado y reutilizable para análisis, Spark y dashboards.

import pandas as pd
import os

##########################################################
# Mostramos las tablas con todas sus columnas seguidas
pd.set_option('display.max_columns', None)      # Mostrar todas las columnas
pd.set_option('display.max_colwidth', None)     # Sin límite en ancho de columna
pd.set_option('display.expand_frame_repr', False)  # No dividir el dataframe en varias líneas
pd.set_option('display.width', 0)               # Ajusta el ancho a la ventana del terminal
##########################################################

# Cargar dataset
# El dataset raw se considera inmutable. Todas las transformaciones se realizan sobre copias para garantizar trazabilidad.
df = pd.read_csv('../data/raw/global_earthquakes_10y.csv')

print(df.head(20))

In [ ]:
## Se priorizan variables relevantes para análisis sísmico y visualización. Métricas técnicas con alto porcentaje de nulos se excluyen para simplificar el modelo.

df = df[['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'place', 'status']]  
print(df.head(20))

In [ ]:
## Normalización de tipos es clave para evitar errores posteriores en agregaciones y procesamiento distribuido.
df['time'] = pd.to_datetime(df['time'], errors='coerce')    
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['mag'] = pd.to_numeric(df['mag'], errors='coerce')   
print(df.dtypes)

In [ ]:
## crear columnas year y month
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month   
print(df.head(20))


In [ ]:
## La categorización de magnitud facilita análisis agregados y visualización para usuarios no técnicos.
def categorize_magnitude(mag):
    if mag < 4.0:
        return 'Low'
    elif 4.0 <= mag < 5.0:
        return 'Moderate'
    elif 5.0 <= mag < 6.0:
        return 'Strong'
    else:
        return 'Major'
df['magnitude_category'] = df['mag'].apply(categorize_magnitude)
print(df.head(20))





In [ ]:
# Guardar dataset limpio
# El dataset procesado se guarda separado del raw para facilitar reproducibilidad y downstream processing.
output_path = '../data/processed/earthquakes_clean.csv'
df.to_csv(output_path, index=False)
print(f'Dataset limpio guardado en: {os.path.abspath(output_path)}')